In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Preprocesamiento del dataset

#### Comentario:

En esta notebook prepararemos el dataset para el desafío de la clase. Al final de esta notebook se genera el archivo 'dataset_preprocessed.csv' que ya se encuentra en la carpeta 'Datos'. 

Esta notebook sirve para repasar limpieza, imputación, manejo de fechas y joins con pandas. Se sugiere hacer la práctica de la clase partiendo del dataset ya preprocesado para ver los temas de feature selection y balance de clases y que esta notebook la revisen fuera del horario de clase.

---

Trabajaremos con un dataset de campañas de marketing. El mismo está representado por un conjunto de tablas relacionales:

* train: relaciona id de consumidores, cupones de descuento, campaña de marketing y contiene la variable target 'redemption_status', que dice si el cupón fue canjeado o no.

* campaign data: fecha de inicio y finalización de la campaña y tipo de campaña

* customer_demographics: rango etareo, estado civil, tamaño de la familia, nro de hijos, nivel de ingresos, propietario/inquilino/hipoteca

* coupon_item_mapping: cada cupón da beneficios sobre distintos items

* item_data: marca, tipo de marca y categoría del item

* customer_transaction_data: datos sobre las compras hechas por los consumidores. No sabemos en qué compra usaron el cupón de descuento, tal vez la promoción sirve para más de una compra.

<img src='../Data/marketing/Schema.png'>

<b>Origen del dataset:</b>

https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [ ]:
train=pd.read_csv('../Data/marketing/train.csv')
cust_dem=pd.read_csv('../Data/marketing/customer_demographics.csv')
trans=pd.read_csv('../Data/marketing/customer_transaction_data.csv')
coupon_item=pd.read_csv('../Data/marketing/coupon_item_mapping.csv')
item_data=pd.read_csv('../Data/marketing/item_data.csv')
campaign_data=pd.read_csv('../Data/marketing/campaign_data.csv')

---
#### Train dataframe

* Mirar el dataframe y chequear si hay valores nulos

---
#### Customer demographics dataframe

* Chequear si hay nulos
* Mirar los dtypes
* Imputar los valores nulos por la moda de las columnas
* Llevar las columnas de variables categóricas a dtype 'category'

---
#### Transactions dataframe
* Chequear si hay nulos
* ver los dtypes
* dropear la columna 'dates'

---
#### Coupon-Item dataframe
* chequear si hay nulos

---
#### Item dataframe
* Chequear si hay nulos
* Llevar las columnas categóricas a dtype 'category'

---
#### Campaign dataframe
* Chequear si hay valores nulos
* Llevar las columnas start_date y end_date a tipo TimeStamp
* Calcular el período de las promos como end_date - start_date
* Dropear start_date y end_date

---
#### Merge Dataframes
1. Hacer un merge entre train y campaign_data. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
2. Hacer un merge de 1. con customer_demographics. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
3. Chequear valores nulos en 2.
4. Imputamos valores faltantes en las columnas demográficas. Para eso, sampleamos con reposición de la distribución conjunta de atributos demográficos observados.
5. Merge con las transacciones promedio por sujeto:
* Hacer un groupby por consumidor en el dataframe de transaccions
* Agregar el valor medio a las variables continuas ('selling_price','quantity', etc)
* merge con el dataframe obtenido en 4. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
6. Merge con transacciones promedio por item
* a. Hacer un groupby por item en el dataframe de transaccions
* b. Calcular el valor medio de las variables continuas
* c. Hacer un merge con el dataframe 'coupon_item'
* d. Hacer un merge de c. con item_data ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
* e. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la moda de las variables categóricas 'brand','brand_type,'category'
* f. Hacer un groupby del dataframe d. agrupando por 'coupon_id'. Para cada grupo calcular la media de las variables continuas.
* g. Hacer un merge de d. con e. y luego con f. ¿Qué columna va en 'on'? ¿Qué criterio va en 'how'?
7. Guardar el dataframe resultante en un csv para la práctica de la clase.